## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

Use the 64-unit or the 1900-unit model?

In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [2]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [7]:
batch_size = 12
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

ValueError: Variable embed_matrix already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/notebooks/unirep.py", line 740, in __init__
    "embed_matrix", dtype=tf.float32, initializer=np.load(os.path.join(self._model_path, "embed_matrix:0.npy"))
  File "<ipython-input-3-f2a9a4323cd4>", line 2, in <module>
    b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [8]:
seq = "MRKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [11]:
from scipy.spatial import distance
a = [1, 2, 3]
b = [4, 5, 6]
dst = distance.euclidean(a, b)
c = np.zeros((4, 6))
print(c)
print(a+c)
print(dst)

[1, 2, 3]
5.196152422706632


In [10]:
from scipy.spatial import distance

def random_line(file_name):
    lines = open(file_name).read().splitlines()
    return np.random.choice(lines).split()

def get_seq(file_name):
    f = open("dataset/fastas/" + file_name + ".fasta", "r")
    next(f)
    seq = ""
    for line in f:
        tmp = line.rstrip()    # Supprimer le "\n"
        seq += tmp
    f.close
    seq_vector = np.array(b.format_seq(seq))
    return seq, seq_vector

f = open("fullProtein.list")

line = next(f).split()
studied_prot = get_seq(line[0])

same_class = []
rd_class = []

nb_seq_class = 5


for i in range (nb_seq_class): #Recuperer les noms de fichiers
    line = next(f).split()
    same_class.append(line[0])
    rd_protein =random_line("fullProtein.list")
    rd_class.append(line[0])
    
f.close

print(same_class)
print(rd_class)

seq_list1 = [] # Recuperer les sequences
dist1 = []
for i in range(nb_seq_class):
    prot = get_seq(same_class[0])
    seq_list1.append(prot)
    #dist1.append(distance.euclidean(studied_prot[1], prot[1]))

seq_list2 = []
dist2 = []
for i in range(nb_seq_class):
    prot = get_seq(rd_class[0])
    seq_list2.append(prot)
    #dist2.append(distance.euclidean(studied_prot[1], prot[1]))

print(seq_list1)
print(seq_list2)

"""rd_protein = random_line("fullProtein.list")
print(random_protein)

file_name = "dataset/fastas/" + protein[0] + ".fasta"
rd_file_name = "dataset/fastas/" + rd_protein[0] + ".fasta"

seq, seq_vector = get_seq(file_name)
rd_seq, rd_seq_vector = get_seq(rd_file_name)

print("Studied sequence :")
print(seq + "\n")
print("Sequence size : " + str(len(seq)) + "\n")
print("Vector representation of the sequence :\n" + str(seq_vector) + "\n")
print("Vector size : " + str(len(seq_vector)) + "\n")
print("Is seq a valide sequence ?\n" + str(b.is_valid_seq(seq)) + "\n")
"""

['d1dlya_', 'd1s69a_', 'd1idra_', 'd1ngka_', 'd1ux8a_']
['d1dlya_', 'd1s69a_', 'd1idra_', 'd1ngka_', 'd1ux8a_']
[('SLFAKLGGREAVEAAVDKFYNKIVADPTVSTYFSNTDMKVQRSKQFAFLAYALGGASEWKGKDMRTAHKDLVPHLSDVHFQAVARHLSDTLTELGVPPEDITDAMAVVASTRTEVLNMPQQ', array([24,  7, 21, 18, 15,  4, 21, 13, 13,  2,  6, 15, 16,  6, 15, 15, 16,
        5,  4, 18, 19,  9,  4, 17, 16, 15,  5, 14,  8, 16,  7,  8, 19, 18,
        7,  9,  8,  5,  1,  4, 16, 10,  2,  7,  4, 10, 18, 15, 18, 21, 15,
       19, 15, 21, 13, 13, 15,  7,  6, 20,  4, 13,  4,  5,  1,  2,  8, 15,
        3,  4,  5, 21, 16, 14,  3, 21,  7,  5, 16,  3, 18, 10, 15, 16, 15,
        2,  3, 21,  7,  5,  8, 21,  8,  6, 21, 13, 16, 14, 14,  6,  5, 17,
        8,  5, 15,  1, 15, 16, 16, 15,  7,  8,  2,  8,  6, 16, 21,  9,  1,
       14, 10, 10])), ('SLFAKLGGREAVEAAVDKFYNKIVADPTVSTYFSNTDMKVQRSKQFAFLAYALGGASEWKGKDMRTAHKDLVPHLSDVHFQAVARHLSDTLTELGVPPEDITDAMAVVASTRTEVLNMPQQ', array([24,  7, 21, 18, 15,  4, 21, 13, 13,  2,  6, 15, 16,  6, 15, 15, 16,
        5,  4

'rd_protein = random_line("fullProtein.list")\nprint(random_protein)\n\nfile_name = "dataset/fastas/" + protein[0] + ".fasta"\nrd_file_name = "dataset/fastas/" + rd_protein[0] + ".fasta"\n\nseq, seq_vector = get_seq(file_name)\nrd_seq, rd_seq_vector = get_seq(rd_file_name)\n\nprint("Studied sequence :")\nprint(seq + "\n")\nprint("Sequence size : " + str(len(seq)) + "\n")\nprint("Vector representation of the sequence :\n" + str(seq_vector) + "\n")\nprint("Vector size : " + str(len(seq_vector)) + "\n")\nprint("Is seq a valide sequence ?\n" + str(b.is_valid_seq(seq)) + "\n")\n'

In [9]:
np.array(b.format_seq(seq))

array([24,  1,  2,  4, 13,  6,  6, 21, 18,  8, 13, 16, 16, 14, 17, 21, 16,
        6, 21,  5, 13,  5, 16,  9, 13,  3,  4, 18,  7, 16,  2, 13,  6, 13,
        6, 13,  5, 15,  8,  9, 13,  4, 21,  8, 21,  4, 18, 17, 11,  8,  8,
       13,  4, 21, 14, 16, 14, 20, 14,  8, 21, 16,  8,  8, 21,  8, 19, 13,
       16, 10, 11, 18, 15,  2, 19, 14,  5,  3,  1,  4, 10,  3,  5, 18, 18,
        4,  7, 15,  1, 14,  6, 13, 19, 16, 10,  6,  2,  8, 17,  7, 18,  4,
        5,  5, 13,  8, 19,  4,  8,  2, 15,  6, 16,  4, 18,  6, 13,  5,  8,
       21, 16,  9,  2, 17,  6, 21,  4, 13, 17,  5, 18,  4,  6,  5, 13,  9,
       17, 21, 13,  3,  4, 21,  6, 19,  9, 18,  9,  7,  3,  9, 16, 19, 17,
        8, 15,  5,  4, 10,  4,  9, 13, 17,  4, 15,  9, 18,  4, 17,  2,  3,
        9, 16,  6,  5, 13,  7, 16, 10, 21, 15,  5,  3, 19, 10, 10,  9,  8,
       14, 17, 13,  5, 13, 14, 16, 21, 21, 14,  5,  9,  3, 19, 21,  7,  8,
       10,  7, 16, 21,  7,  4,  5, 14,  9,  6,  4,  2,  5,  3,  1, 16, 21,
       21,  6, 18, 16,  8

We also provide a function that will check your amino acid sequences don't contain any characters which will break the UniRep model.

In [ ]:
b.is_valid_seq(seq)

You could use your own data flow as long as you ensure that the data format is obeyed. Alternatively, you can use the data flow we've implemented for UniRep training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. Here's how to do that:

First, sequences need to be saved in the correct format. Suppose we have a new-line seperated file of amino acid sequences, `seqs.txt`, and we want to format them. Note that training is currently only publicly supported for amino acid sequences less than 275 amino acids as gradient updates for sequences longer than that start to get unwieldy. If you want to train on sequences longer than this, please reach out to us. 

Sequence formatting can be done as follows:

In [154]:
# Before you can train your model, 
with open("seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

This is what the integer format looks like

In [73]:
!head -n1 formatted.txt

24,1,2,4,13,6,6,21,18,8,13,16,16,14,17,21,16,6,21,5,13,5,16,9,13,3,4,18,7,16,2,13,6,13,6,13,5,15,8,9,13,4,21,8,21,4,18,17,11,8,8,13,4,21,14,16,14,20,14,8,21,16,8,8,21,8,19,13,16,10,11,18,15,2,19,14,5,3,1,4,10,3,5,18,18,4,7,15,1,14,6,13,19,16,10,6,2,8,17,7,18,4,5,5,13,8,19,4,8,2,15,6,16,4,18,6,13,5,8,21,16,9,2,17,6,21,4,13,17,5,18,4,6,5,13,9,17,21,13,3,4,21,6,19,9,18,9,7,3,9,16,19,17,8,15,5,4,10,4,9,13,17,4,15,9,18,4,17,2,3,9,16,6,5,13,7,16,10,21,15,5,3,19,10,10,9,8,14,17,13,5,13,14,16,21,21,14,5,9,3,19,21,7,8,10,7,16,21,7,4,5,14,9,6,4,2,5,3,1,16,21,21,6,18,16,8,15,15,13,17,8,3,13,1,5,6,21,19,4


Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training UniRep representations.

Now we can use a custom function to bucket, batch and pad sequences from `formatted.txt` (which has the correct integer codex after calling `babbler.format_seq()`). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. The interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.

This is also doing a few other things:
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset

In [26]:
bucket_op = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval

AttributeError: 'list' object has no attribute 'bucket_batch_pad'

Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the `bucket_op`. Please reach out if you have questions on how to do this.

Now that we have the `bucket_op`, we can simply `sess.run()` it to get a correctly formatted batch

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    
print(batch)
print(batch.shape)

[[24  1  7 ...  0  0  0]
 [24  1 10 ...  4  2 19]
 [24  1 13 ...  0  0  0]
 ...
 [24  1  2 ...  0  0  0]
 [24  1 17 ...  0  0  0]
 [24  1 15 ...  0  0  0]]
(12, 248)


You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

## Training a top model and a top model + mLSTM.

First, obtain all of the ops needed to output a representation

In [159]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

`final_hidden` should be a batch_size x rep_dim matrix.

Lets say we want to train a basic feed-forward network as the top model, doing regression with MSE loss, and the Adam optimizer. We can do that by:

1.  Defining a loss function.

2.  Defining an optimizer that's only optimizing variables in the top model.

3.  Minimizing the loss inside of a TensorFlow session

In [160]:
y_placeholder = tf.placeholder(tf.float32, shape=[None,1], name="y")
initializer = tf.contrib.layers.xavier_initializer(uniform=False)

with tf.variable_scope("top"):
    prediction = tf.contrib.layers.fully_connected(
        final_hidden, 1, activation_fn=None, 
        weights_initializer=initializer,
        biases_initializer=tf.zeros_initializer()
    )

loss = tf.losses.mean_squared_error(y_placeholder, prediction)

You can specifically train the top model first by isolating variables of the "top" scope, and forcing the optimizer to only optimize these.

In [161]:
learning_rate=.001
top_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")
optimizer = tf.train.AdamOptimizer(learning_rate)
top_only_step_op = optimizer.minimize(loss, var_list=top_variables)
all_step_op = optimizer.minimize(loss)

We next need to define a function that allows us to calculate the length each sequence in the batch so that we know what index to use to obtain the right "final" hidden state

In [162]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([ 87, 111, 223, 148, 124, 195, 176, 232, 205,  97, 239, 261])

We are ready to train. As an illustration, let's learn to predict the number 42 just optimizing the top model.

In [163]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, top_only_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
                  
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 1741.5738525390625
Iteration 1: 1739.5284423828125
Iteration 2: 1737.1976318359375
Iteration 3: 1735.5791015625
Iteration 4: 1733.2252197265625
Iteration 5: 1736.0694580078125
Iteration 6: 1733.9964599609375
Iteration 7: 1731.6090087890625
Iteration 8: 1729.8792724609375
Iteration 9: 1727.7677001953125


We can also jointly train the top model and the mLSTM. Note that if using the 1900-unit (full) model, you will need a GPU with at least 16GB RAM. To see a demonstration of joint training with fewer computational resources, please run this notebook using the 64-unit model.

In [17]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, all_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
        
        print("Iteration {0}: {1}".format(i,loss_))

Iteration 0: 1757.400390625
Iteration 1: 1720.1832275390625
Iteration 2: 1650.4283447265625
Iteration 3: 1552.2196044921875
Iteration 4: 1369.4599609375
Iteration 5: 1174.6700439453125
Iteration 6: 981.3558959960938
Iteration 7: 798.6965942382812
Iteration 8: 715.9775390625
Iteration 9: 580.2219848632812
